# Treinando um modelo de linguagem

Original: https://github.com/huggingface/transformers/tree/master/examples/language-modeling

Instalando 🤗 Transformers e 🤗 Datasets

In [ ]:
! pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Preparando o dataset

In [ ]:

#datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

from datasets import load_dataset


ds = load_dataset("deokhk/pt_wiki_sentences_1000000")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

(…)-00000-of-00001-b336e175ca62a9af.parquet:   0%|          | 0.00/89.2M [00:00<?, ?B/s]

(…)-00000-of-00001-6e4a277958acc0b7.parquet:   0%|          | 0.00/80.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Qualquer dataset hospedado no HF pode vir aqui ([do hub](https://huggingface.co/datasets)). Você também pode usar o seu dataset, que também pode estar em json ou csv ([documentação](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) )

In [ ]:
# se quiser usar o seu
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

Acessando os dados

In [ ]:
ds["train"][5]

{'sentence': 'Ele foi julgado à parte das outras acusadas pelos homicídios, sendo condenado à morte em 21 de outubro de 1971, escapando da execução quando a Suprema Corte da Califórnia aboliu a pena de morte no estado em 1972, passando a cumprir prisão perpétua.'}

In [ ]:
ds['train'].shape

(1000000, 1)

In [ ]:
ds['train'] = ds['train'].select(range(5000))

In [ ]:
ds['train'].shape

(5000, 1)

In [ ]:
ds['dev'].shape

(1000, 1)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 5000
    })
    dev: Dataset({
        features: ['sentence'],
        num_rows: 1000
    })
})

Dando uma olhadinha no dataset:

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(ds["train"])

,sentence
0,A sigla BPM pode significar:
1,Site Observit Portfólio de projectos Ver também.
2,"Número muito próximo de dois anos atrás de acordo com as pesquisas de 2017, evidenciando o desafio que é garantir a proteção e a segurança para as mulheres."
3,"Jamais me senti tão infeliz!""."
4,"Durante a competição mundial, a atleta de dezesseis anos, competiu no individual geral e atingiu um total de 56,100 pontos, terminando assim na 37ª posição."
5,Esta última missão foi considerada a mais tranquila de toda a 'Fase Um' e nela Andrews realizou 27 pesquisas científicas na área da tecnologia avançada.
6,"No interior do CN existe também uma boa estrutura médica, com enfermaria, farmácia e centros odontológico e de fisioterapia."
7,"Estende-se por uma área de 25,13 km²."
8,"Ele também foi à Inglaterra, à Irlanda, e à França, adquirindo navios e suprimentos para a Confederação."
9,"Para o uso melhor destes atualmente, não se sabe se pode haver um procedimento criptanalítico que possa inverter (ou inverter parcialmente) estas transformações sem saber a chave usada durante a criptografia."


## Causal Language modeling (autorregressiva)

Vamos concatenar os exemplos depois de tokenizados e organiza-los como abaixo
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```

Os rótulos são as entradas, com um shift para a esquerda.
Vamos usar a arquitetura do GPT2 ([`gpt2`](https://huggingface.co/gpt2) )

In [ ]:
model_checkpoint = "pierreguillou/gpt2-small-portuguese"
tokenizer_checkpoint = "pierreguillou/gpt2-small-portuguese"

O tokenizador precisa ser treinado antes, para considerar subtokens de acordo com a frequencia deles

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenizando o texto com  [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"])

Aplicando a função no dataset inteiro, com 4 processos para acelerar.

In [ ]:
tokenized_datasets = ds.map(tokenize_function, batched=True, num_proc=4, remove_columns=["sentence"])

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["dev"][1]

{'input_ids': [2704,
  3739,
  9987,
  367,
  300,
  1738,
  544,
  3184,
  358,
  8724,
  3916,
  300,
  20250,
  261,
  1565,
  261,
  1961,
  12,
  8844,
  330,
  29417,
  260,
  627,
  11463,
  474,
  347,
  2966,
  489,
  2325,
  72,
  14],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Se os seus exemplos forem grandes, ou compostos de mais de uma sentença. Caso contrário, não precisa.
Vamos concatenar o texto em pedaços de 32

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 32

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Observe que os labels são os próprios textos. A biblioteca do Transformers vai fazer o shift para a direita.

O batch será de 1000 exemplos X tamanho do bloco

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# caso nao precise agrupar
lm_datasets = tokenized_datasets

In [ ]:
ds["train"][5]

{'sentence': 'Ele foi julgado à parte das outras acusadas pelos homicídios, sendo condenado à morte em 21 de outubro de 1971, escapando da execução quando a Suprema Corte da Califórnia aboliu a pena de morte no estado em 1972, passando a cumprir prisão perpétua.'}

Podemos ver alterações no dataset (mas lembre que convertemos para índices e tokens, precisamos recuperar o texto com decode)

In [ ]:
tokenizer.decode(lm_datasets["train"][5]["input_ids"])

' 1969, em Los Angeles.Biografia.Watson cometeu os crimes em companhia de Susan Atkins, Leslie Van Houten e Patricia Krenwinkel,'

Vamos criar as variáveis a partir do checkpoint do modelo

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM

# inicialização aleatória
#config = AutoConfig.from_pretrained(model_checkpoint)
#
#model = AutoModelForCausalLM.from_config(config)

# carregamento do checkpoint
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
def gen(prompt, model):
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  # model output using Top-k sampling text generation method
  sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True,
                                max_length=50, # put the token number you want
                                top_k=40,
                                num_return_sequences=1)

  # generated sequence
  for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

In [ ]:
# input sequence
prompt = "Quem era Susan Atkins? Susan Atkins, era uma"
gen(prompt, model)

>> Generated text 1

Quem era Susan Atkins? Susan Atkins, era um estudante de história no Texas que viajou muito e, tendo se mudado de uma residência em St. Louis, teve como filha, Susan DeMann.Mais tarde, Susan mudou-se para


Definindo hiperparâmetros do treinamento

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    f"{model_checkpoint}-wiki_sentences",
    eval_strategy = "epoch",
    #num_train_epochs =
    learning_rate=2e-5,
    weight_decay=0.01
)

Classe de treino

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["dev"],
)

And we can train our model:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.239800,3.968158
2,3.957200,3.995295
3,3.829400,4.011760


TrainOutput(global_step=1704, training_loss=3.9825645410958592, metrics={'train_runtime': 311.773, 'train_samples_per_second': 43.686, 'train_steps_per_second': 5.466, 'total_flos': 222424842240000.0, 'train_loss': 3.9825645410958592, 'epoch': 3.0})

Avaliaçao do modelo treinado

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 55.24


In [ ]:
# input sequence
prompt = "Quem era Susan Atkins? Susan Atkins, era uma"
gen(prompt, model)

>> Generated text 1

Quem era Susan Atkins? Susan Atkins, era uma mulher católica de letras que se casa com o viúvo Fred White, e que, por estar grávida, não possuía filhos.Eles iniciaram uma discussão sobre se poderia obter ajuda após a gravidez de Susan


## Masked language modeling

Vamos usar o mesmo preprocessamento, mas vamos mascarar alguns tokens usando `[MASK]` e os labels serão ajustados para incluirem somente os tokens mascarados (quem não é mascarado, não precisa entrar na predição).

Vamos usar o BERTimbau

In [ ]:
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer_checkpoint = "neuralmind/bert-base-portuguese-cased"

Atualizando a tokenização



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = ds.map(tokenize_function, batched=True, num_proc=4, remove_columns=["sentence"])

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Agrupando, caso necessário

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
# caso nao precise agrupar
lm_datasets = tokenized_datasets

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = model_checkpoint

# para treinar do zero
#model = AutoModelForMaskedLM.from_config(config)

We redefine our `TrainingArguments`:

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Organizando o dataset em batches nos tensores e colocando as máscaras.

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["dev"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,8.253100,7.731552
2,7.525700,7.576919
3,7.318500,7.389372


TrainOutput(global_step=1875, training_loss=7.626073697916667, metrics={'train_runtime': 438.1866, 'train_samples_per_second': 34.232, 'train_steps_per_second': 4.279, 'total_flos': 528152982376800.0, 'train_loss': 7.626073697916667, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1681.40


In [ ]:
from transformers import pipeline
pipe = pipeline('fill-mask', model=model, tokenizer=tokenizer, device='cuda:0')

pipe('Tinha uma [MASK] no meio do caminho.')

[{'score': 0.0538550540804863,
  'token': 125,
  'token_str': 'de',
  'sequence': 'Tinha uma de no meio do caminho.'},
 {'score': 0.04451660439372063,
  'token': 117,
  'token_str': ',',
  'sequence': 'Tinha uma, no meio do caminho.'},
 {'score': 0.01995236426591873,
  'token': 123,
  'token_str': 'a',
  'sequence': 'Tinha uma a no meio do caminho.'},
 {'score': 0.0169720146805048,
  'token': 122,
  'token_str': 'e',
  'sequence': 'Tinha uma e no meio do caminho.'},
 {'score': 0.014299420639872551,
  'token': 180,
  'token_str': 'da',
  'sequence': 'Tinha uma da no meio do caminho.'}]